In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("spark://spark-master:7077").getOrCreate()
sc = spark.sparkContext

In [88]:
df = spark.read.options(header='True', inferSchema='True', delimiter=',').csv("/home/jovyan/notebooks/energy_hour.csv")

In [4]:
df.count()

21652

In [5]:
df.first()

Row(id=90, created_at=datetime.datetime(2022, 3, 12, 7, 0), reference='takurika1', power_real=-2790.72, power_counter=12, power_real_temp=-33488.6, pac_sum=351.794, pac_sum_counter=11, pac_sum_temp=3869.73, total_energy=11716300, co2factor=1925920.0, update_at='2022-03-12 12:59:55', day_energy=None)

In [6]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- reference: string (nullable = true)
 |-- power_real: double (nullable = true)
 |-- power_counter: integer (nullable = true)
 |-- power_real_temp: double (nullable = true)
 |-- pac_sum: double (nullable = true)
 |-- pac_sum_counter: integer (nullable = true)
 |-- pac_sum_temp: double (nullable = true)
 |-- total_energy: integer (nullable = true)
 |-- co2factor: double (nullable = true)
 |-- update_at: string (nullable = true)
 |-- day_energy: integer (nullable = true)



In [11]:
df.show(5)

+---+-------------------+-----------+----------+-------------+---------------+-------+---------------+------------+------------+---------+-------------------+----------+
| id|         created_at|  reference|power_real|power_counter|power_real_temp|pac_sum|pac_sum_counter|pac_sum_temp|total_energy|co2factor|          update_at|day_energy|
+---+-------------------+-----------+----------+-------------+---------------+-------+---------------+------------+------------+---------+-------------------+----------+
| 90|2022-03-12 07:00:00|  takurika1|  -2790.72|           12|       -33488.6|351.794|             11|     3869.73|    11716300|1925920.0|2022-03-12 12:59:55|      null|
| 91|2022-03-12 07:00:00|comfenalco1|   59.7637|           12|        717.164| 562.77|             11|     6190.47|    27803800|4570390.0|2022-03-12 12:59:54|      null|
| 92|2022-03-12 07:00:00|       edlv|    3563.2|            2|         7126.4|    0.0|              0|         0.0|           0|      0.0|2022-03-12 1

In [12]:
df.filter(df.day_energy > 0).count()

13479

In [24]:
df.filter(df.day_energy > 0).show()

+---+-------------------+-----------+----------+-------------+---------------+-------+---------------+------------+------------+---------+-------------------+----------+
| id|         created_at|  reference|power_real|power_counter|power_real_temp|pac_sum|pac_sum_counter|pac_sum_temp|total_energy|co2factor|          update_at|day_energy|
+---+-------------------+-----------+----------+-------------+---------------+-------+---------------+------------+------------+---------+-------------------+----------+
|103|1999-12-31 19:00:00|  takurika1|   2516.92|            3|        25615.9|    0.0|              2|         0.0|    29104900|4784260.0|2022-10-21 21:57:11|     51432|
|231|2022-03-14 06:00:00|  takurika1|  -271.995|          360|       -97918.0|2.42898|            360|     874.433|    11845300|1947130.0|2022-03-14 06:59:54|       360|
|232|2022-03-14 06:00:00|comfenalco1|   18844.3|          360|      6783930.0|9.38431|            360|     3378.35|    28029600|4607510.0|2022-03-14 0

In [15]:
df.filter(df.day_energy > 0).groupBy("reference").count().sort("count",ascending=False).show()

+-----------+-----+
|  reference|count|
+-----------+-----+
|comfenalco1| 4439|
|  takurika1| 4153|
|  takurika2| 2809|
|       edlv| 2047|
|  takurika3|   31|
+-----------+-----+



In [18]:
df.describe('total_energy').show()

+-------+--------------------+
|summary|        total_energy|
+-------+--------------------+
|  count|               21652|
|   mean|2.4223917295399964E7|
| stddev| 1.899760677268646E7|
|    min|                   0|
|    max|            70324400|
+-------+--------------------+



In [19]:
df.describe().show()

+-------+-----------------+-----------+------------------+------------------+------------------+-----------------+-----------------+-----------------+--------------------+------------------+-------------------+-----------------+
|summary|               id|  reference|        power_real|     power_counter|   power_real_temp|          pac_sum|  pac_sum_counter|     pac_sum_temp|        total_energy|         co2factor|          update_at|       day_energy|
+-------+-----------------+-----------+------------------+------------------+------------------+-----------------+-----------------+-----------------+--------------------+------------------+-------------------+-----------------+
|  count|            21652|      21652|             21652|             21652|             21652|            21652|            21652|            21652|               21652|             21652|              21637|            18602|
|   mean|34789.40421208202|       null|12310.708827065848| 2.704276741178644| 3862.4

In [23]:
df.createOrReplaceTempView("ENERGY")
gdf = spark.sql("SELECT reference, count(*) as number from ENERGY where day_energy >0 group by reference sort by number desc")
gdf.show()

+-----------+------+
|  reference|number|
+-----------+------+
|comfenalco1|  4439|
|  takurika1|  4153|
|  takurika2|  2809|
|       edlv|  2047|
|  takurika3|    31|
+-----------+------+



In [45]:
groupDataframe3 = spark.sql("SELECT CAST(created_at AS DATE), reference, count(*) from ENERGY  where day_energy >0 group by CAST(created_at AS DATE), reference").sort("created_at",ascending=False)

In [46]:
groupDataframe3.show()

+----------+-----------+--------+
|created_at|  reference|count(1)|
+----------+-----------+--------+
|2022-11-07|  takurika2|      11|
|2022-11-07|  takurika3|      12|
|2022-11-07|  takurika1|      16|
|2022-11-07|       edlv|      17|
|2022-11-07|comfenalco1|      17|
|2022-11-06|       edlv|      17|
|2022-11-06|  takurika2|      10|
|2022-11-06|comfenalco1|      17|
|2022-11-06|  takurika3|      11|
|2022-11-06|  takurika1|      16|
|2022-11-05|  takurika1|      18|
|2022-11-05|       edlv|      18|
|2022-11-05|comfenalco1|      19|
|2022-11-05|  takurika3|       8|
|2022-11-05|  takurika2|       6|
|2022-11-04|  takurika1|      18|
|2022-11-04|comfenalco1|      19|
|2022-11-04|       edlv|      18|
|2022-11-03|  takurika1|      19|
|2022-11-03|       edlv|      18|
+----------+-----------+--------+
only showing top 20 rows



In [67]:
df.createOrReplaceTempView("ENERGY")
groupDataframe4 = spark.sql("SELECT CAST(created_at AS DATE), reference, count(*) from ENERGY where reference like 'takurika1' and day_energy is null group by CAST(created_at AS DATE), reference").sort("created_at",ascending=False)

In [68]:
groupDataframe4.show(20)

+----------+---------+--------+
|created_at|reference|count(1)|
+----------+---------+--------+
|2022-10-31|takurika1|       1|
|2022-10-30|takurika1|       4|
|2022-10-29|takurika1|       1|
|2022-10-17|takurika1|       2|
|2022-10-16|takurika1|       2|
|2022-10-03|takurika1|       1|
|2022-09-29|takurika1|       1|
|2022-09-28|takurika1|       2|
|2022-09-26|takurika1|       1|
|2022-09-25|takurika1|       1|
|2022-09-24|takurika1|       1|
|2022-09-23|takurika1|       1|
|2022-09-22|takurika1|       1|
|2022-09-21|takurika1|       6|
|2022-07-13|takurika1|       2|
|2022-07-12|takurika1|       3|
|2022-03-14|takurika1|       4|
|2022-03-13|takurika1|      24|
|2022-03-12|takurika1|      16|
+----------+---------+--------+



In [70]:
groupDataframe4.groupby('created_at','reference').agg({'count(1)': 'mean'}).sort("created_at",ascending=False).show(40)

+----------+---------+-------------+
|created_at|reference|avg(count(1))|
+----------+---------+-------------+
|2022-10-31|takurika1|          1.0|
|2022-10-30|takurika1|          4.0|
|2022-10-29|takurika1|          1.0|
|2022-10-17|takurika1|          2.0|
|2022-10-16|takurika1|          2.0|
|2022-10-03|takurika1|          1.0|
|2022-09-29|takurika1|          1.0|
|2022-09-28|takurika1|          2.0|
|2022-09-26|takurika1|          1.0|
|2022-09-25|takurika1|          1.0|
|2022-09-24|takurika1|          1.0|
|2022-09-23|takurika1|          1.0|
|2022-09-22|takurika1|          1.0|
|2022-09-21|takurika1|          6.0|
|2022-07-13|takurika1|          2.0|
|2022-07-12|takurika1|          3.0|
|2022-03-14|takurika1|          4.0|
|2022-03-13|takurika1|         24.0|
|2022-03-12|takurika1|         16.0|
+----------+---------+-------------+



In [33]:
from pyspark.sql.functions import split

In [89]:
df = df.withColumn('fecha', split(df['created_at'], ' ').getItem(0)).withColumn('hora', split(df['created_at'], ' ').getItem(1))

In [90]:
df.createOrReplaceTempView("ENERGY")

In [97]:
groupDataframe5 = spark.sql("SELECT fecha, reference, hora, count(*) from ENERGY where reference like 'takurika1' and day_energy is null group by fecha, reference, hora").sort("fecha",ascending=False)

In [98]:
groupDataframe5.show(50)

+----------+---------+--------+--------+
|     fecha|reference|    hora|count(1)|
+----------+---------+--------+--------+
|2022-10-31|takurika1|00:00:00|       1|
|2022-10-30|takurika1|06:00:00|       1|
|2022-10-30|takurika1|01:00:00|       1|
|2022-10-30|takurika1|11:00:00|       1|
|2022-10-30|takurika1|15:00:00|       1|
|2022-10-29|takurika1|19:00:00|       1|
|2022-10-17|takurika1|05:00:00|       1|
|2022-10-17|takurika1|03:00:00|       1|
|2022-10-16|takurika1|03:00:00|       1|
|2022-10-16|takurika1|10:00:00|       1|
|2022-10-03|takurika1|06:00:00|       1|
|2022-09-29|takurika1|00:00:00|       1|
|2022-09-28|takurika1|14:00:00|       1|
|2022-09-28|takurika1|21:00:00|       1|
|2022-09-26|takurika1|01:00:00|       1|
|2022-09-25|takurika1|23:00:00|       1|
|2022-09-24|takurika1|11:00:00|       1|
|2022-09-23|takurika1|05:00:00|       1|
|2022-09-22|takurika1|00:00:00|       1|
|2022-09-21|takurika1|12:00:00|       1|
|2022-09-21|takurika1|19:00:00|       1|
|2022-09-21|taku

In [79]:
groupDataframe5.groupby('fecha','reference').agg({'count(1)': 'mean'}).sort("hora",ascending=False).show(40)

AnalysisException: Cannot resolve column name "count(1)" among (id, created_at, reference, power_real, power_counter, power_real_temp, pac_sum, pac_sum_counter, pac_sum_temp, total_energy, co2factor, update_at, day_energy, fecha, hora)

DataFrame[id: int, created_at: timestamp, reference: string, power_real: double, power_counter: int, power_real_temp: double, pac_sum: double, pac_sum_counter: int, pac_sum_temp: double, total_energy: int, co2factor: double, update_at: string, day_energy: int]

In [93]:
groupDataframe2 = spark.sql("SELECT gender, avg(salary) from PERSON_DATA group by gender")
groupDataframe2.show()

+------+------------------+
|gender|       avg(salary)|
+------+------------------+
|     M|3666.6666666666665|
|     F|            1999.5|
+------+------------------+



In [49]:
dataframe.groupBy("gender").count().sort("count",ascending=False).show()

+------+-----+
|gender|count|
+------+-----+
|     M|    3|
|     F|    2|
+------+-----+



In [50]:
dataframe.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [54]:
dataframe.filter(dataframe.salary > 3999).count()

3

In [56]:
dataframe.groupby('gender').agg({'salary': 'mean'}).show()

+------+------------------+
|gender|       avg(salary)|
+------+------------------+
|     M|3666.6666666666665|
|     F|            1999.5|
+------+------------------+

